In [1]:
 pip install --upgrade pip

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 2.1 MB 102.6 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
! pip install jieba

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 114.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314458 sha256=9a814f157dc855e00219fc1353486f6227038543e89fe5e35b25a5a3a9dc73c6
  Stored in directory: /home/ma-user/.cache/pip/wheels/86/bc/97/67c05f24b07573fd425039f944f8bc57c8dbc6f2c0bcc046fa
Successfully built jieba
DEPRECATION: moxing-framework 2.1.0.5d9c87c8 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import jieba
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
train_data = pd.read_csv('data/train.tsv', sep='\t')
valid_data = pd.read_csv('data/dev.tsv', sep='\t')
test_data = pd.read_csv('data/test.tsv', sep='\t') 
x_train, y_train = train_data.text_a.values, train_data.label.values # 训练集
x_valid, y_valid = valid_data.text_a.values, valid_data.label.values # 验证集
x_test, y_test = test_data.text_a.values, test_data.label.values # 测试集

2025-04-28 10:36:37.696781: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2025-04-28 10:36:37.698227: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
/home/ma-user/anaconda3/envs/TensorFlow-2.1/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
train_data

,text_a,label
0,选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全...,1
1,15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很...,1
2,房间太小。其他的都一般。。。。。。。。。,0
3,"1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸...",0
4,"今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量...",1
...,...,...
9141,看过该书，感觉中医暂时不会消亡，尚有一、二十株老树活着，还有毛以林、黄煌、刘力红等一批有一定...,1
9142,这本书没读到底，不是特别喜欢。完全可以用序中的评价来表达我的感受：可以包容，却不想实践。除了...,0
9143,"虽是观景房,不过我住的楼层太低(19楼)看不到江景,但地点很好,离轻轨临江门站和较场口站(起...",1
9144,性价比不错，交通方便。行政楼层感觉很好，只是早上8点楼上装修，好吵。 中餐厅档次太低，虽然便...,1


In [7]:
x_train, y_train

(array(['选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
        '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错',
        '房间太小。其他的都一般。。。。。。。。。', ...,
        '虽是观景房,不过我住的楼层太低(19楼)看不到江景,但地点很好,离轻轨临江门站和较场口站(起点)很近,解放碑就在附近(大约100多公尺吧)!',
        '性价比不错，交通方便。行政楼层感觉很好，只是早上8点楼上装修，好吵。 中餐厅档次太低，虽然便宜，但是和酒店档次不相配。',
        '跟心灵鸡汤没什么本质区别嘛，至少我不喜欢这样读经典，把经典都解读成这样有点去中国化的味道了'], dtype=object),
 array([1, 1, 0, ..., 1, 1, 0]))

In [8]:
vocab = set()
cut_docs = train_data.text_a.apply(lambda x: jieba.cut(x)).values
for doc in cut_docs:
    for word in doc:
        if word.strip():
            vocab.add(word.strip())

# 将词表写入本地vocab.txt文件
with open('data/vocab.txt', 'w') as file:
    for word in  vocab:
        file.write(word)
        file.write('\n')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.661 seconds.
Prefix dict has been built successfully.


In [9]:
class Config():
    embedding_dim = 300 # 词向量维度
    max_seq_len = 200 # 文章最大词数
    vocab_file = 'data/vocab.txt' # 词汇表文件路径
config = Config()

In [10]:
class Preprocessor():
    def __init__(self, config):
        self.config = config
        # 初始化词和id的映射词典，预留0给padding字符，1给词表中未见过的词
        token2idx = {"[PAD]": 0, "[UNK]": 1} # {word：id}
        with open(config.vocab_file, 'r') as reader:
            for index, line in enumerate(reader):
                token = line.strip()
                token2idx[token] = index+2
                
        self.token2idx = token2idx
        
    def transform(self, text_list):
        # 文本分词，并将词转换成相应的id, 最后不同长度的文本padding长统一长度，后面补0
        idx_list = [[self.token2idx.get(word.strip(), self.token2idx['[UNK]']) for word in jieba.cut(text)] for text in text_list]
        idx_padding = pad_sequences(idx_list, self.config.max_seq_len, padding='post')
        
        return idx_padding

In [11]:
preprocessor = Preprocessor(config)
preprocessor.transform(['性价比不错，交通方便。', '房间太小。其他的都一般。'])

array([[25914,  5479,  3103, 16444, 10064, 18169,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [12]:
from tensorflow.keras import regularizers
class TextCNN(object):
    def __init__(self, config):
        self.config = config
        self.preprocessor = Preprocessor(config)
        self.class_name = {0: '负面', 1: '正面'}
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)  # 降低学习率

    def build_model(self):
        # 输入层
        idx_input = tf.keras.layers.Input((self.config.max_seq_len,))
        
        # 嵌入层（增加trainable参数控制）
        input_embedding = tf.keras.layers.Embedding(
            len(self.preprocessor.token2idx),
            self.config.embedding_dim,
            input_length=self.config.max_seq_len,
            mask_zero=True,
            embeddings_regularizer=regularizers.l2(1e-4)  # 添加嵌入层正则化
        )(idx_input)

        # 多尺度卷积层（减少滤波器数量）
        conv_blocks = []
        for kernel_size in [3, 4, 5]:
            conv = tf.keras.layers.Conv1D(
                filters=64,  # 减少滤波器数量
                kernel_size=kernel_size,
                activation='relu',
                padding='valid',
                kernel_regularizer=regularizers.l2(1e-4)  # 卷积核正则化
            )(input_embedding)
            pool = tf.keras.layers.GlobalMaxPooling1D()(conv)
            conv_blocks.append(pool)

        # 特征拼接
        concat = tf.keras.layers.concatenate(conv_blocks, axis=-1)
        
        # 添加批标准化层
        bn = tf.keras.layers.BatchNormalization()(concat)
        
        # 全连接层（减少神经元数量）
        dense = tf.keras.layers.Dense(
            64,  # 减少神经元数量
            activation='relu',
            kernel_regularizer=regularizers.l2(0.01)  # 增强L2正则化
        )(bn)
        
        # 增强Dropout
        dropout = tf.keras.layers.Dropout(0.7)(dense)  # 提高dropout比例
        
        # 输出层
        output = tf.keras.layers.Dense(2, activation='softmax')(dropout)

        # 编译模型
        model = tf.keras.Model(inputs=idx_input, outputs=output)
        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=self.optimizer,  # 使用自定义优化器
            metrics=['accuracy']
        )
        model.summary()
        self.model = model

    def fit(self, x_train, y_train, x_valid=None, y_valid=None, epochs=50, batch_size=64, **kwargs):
    # 初始化回调
        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',  # 改为监控准确率
            patience=10,  # 增大耐心值
            mode='max',
            restore_best_weights=True
        )
    
        lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6,
            verbose=1
        )

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            './improved_cnn_checkpoint',
            save_weights_only=True,
            save_best_only=True,
            monitor='val_accuracy'
        )

        # 数据预处理
        self.build_model()
        x_train = self.preprocessor.transform(x_train)
        valid_data = (self.preprocessor.transform(x_valid), y_valid) if x_valid is not None else None

        # 合并显式传递的回调和 kwargs 中的回调
        callbacks = [early_stop, lr_scheduler, checkpoint] + kwargs.get('callbacks', [])

        # 过滤掉 kwargs 中的 callbacks 参数
        filtered_kwargs = {k: v for k, v in kwargs.items() if k != 'callbacks'}

        # 开始训练
        history = self.model.fit(
            x=x_train,
            y=y_train,
            validation_data=valid_data,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=callbacks,  # 显式传递合并后的回调
            **filtered_kwargs  # 过滤掉 callbacks 后的其他参数
        )
        return history
    # 保持其他方法不变
    def evaluate(self, x_test, y_test):
        x_test = self.preprocessor.transform(x_test)
        y_pred = np.argmax(self.model.predict(x_test), axis=-1)
        print(classification_report(y_test, y_pred, target_names=['负面', '正面']))
    
    def single_predict(self, text):
        x = self.preprocessor.transform([text])
        prob = self.model.predict(x)[0]
        label_idx = np.argmax(prob)
        return self.class_name[label_idx], prob[label_idx]
    
    def load_model(self, ckpt_file):
        self.build_model()
        self.model.load_weights(ckpt_file)

In [13]:
# ========== 修改后的回调函数设置 ==========
# 1. 增强早停策略
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # 改为监控验证准确率
    patience=10,            # 增大耐心值
    mode='max',             # 指标越大越好
    restore_best_weights=True  # 自动恢复最佳权重
)

# 2. 添加学习率调度
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',     # 监控验证损失
    factor=0.5,             # 学习率衰减系数
    patience=3,             # 连续3次未改进则调整
    min_lr=1e-6             # 最小学习率
)

# 3. 优化模型保存策略
checkpoint_prefix = './checkpoints/textcnn_imdb_ckpt'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',  # 与早停监控指标一致
    mode='max'               # 保存准确率最高的模型
)

# ========== 初始化模型和训练 ==========
textcnn = TextCNN(config)
textcnn.fit(
    x_train, y_train,
    x_valid=x_valid, 
    y_valid=y_valid,
    epochs=50,
    callbacks=[early_stop, checkpoint_callback, lr_scheduler]  # 添加学习率调度
)

2025-04-28 10:44:23.268357: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/lib/x86_64-linux-gnu/
2025-04-28 10:44:23.268394: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2025-04-28 10:44:23.268413: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (notebook-936bea3e-c2f3-4af8-a777-65a410cbc27a): /proc/driver/nvidia/version does not exist
2025-04-28 10:44:23.268591: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2025-04-28 10:44:23.277909: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 300)     10527900    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 198, 64)      57664       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 197, 64)      76864       embedding[0][0]                  
______________________________________________________________________________________________

/home/ma-user/anaconda3/envs/TensorFlow-2.1/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/ma-user/anaconda3/envs/TensorFlow-2.1/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


9146/9146 [==============================] - 99s 11ms/sample - loss: 2.4471 - accuracy: 0.5681 - val_loss: 2.2311 - val_accuracy: 0.6650
Epoch 2/50
9146/9146 [==============================] - 97s 11ms/sample - loss: 2.0220 - accuracy: 0.7339 - val_loss: 2.0770 - val_accuracy: 0.8142
Epoch 3/50
9146/9146 [==============================] - 97s 11ms/sample - loss: 1.7811 - accuracy: 0.8326 - val_loss: 1.9005 - val_accuracy: 0.8400
Epoch 4/50
9146/9146 [==============================] - 98s 11ms/sample - loss: 1.5977 - accuracy: 0.8883 - val_loss: 1.7082 - val_accuracy: 0.8567
Epoch 5/50
9146/9146 [==============================] - 97s 11ms/sample - loss: 1.4554 - accuracy: 0.9283 - val_loss: 1.5585 - val_accuracy: 0.8700
Epoch 6/50
9146/9146 [==============================] - 96s 10ms/sample - loss: 1.3315 - accuracy: 0.9574 - val_loss: 1.4598 - val_accuracy: 0.8750
Epoch 7/50
9146/9146 [==============================] - 95s 10ms/sample - loss: 1.2214 - accuracy: 0.9718 - val_loss: 1.383

In [14]:
textcnn.evaluate(x_test, y_test) # 测试集评估

              precision    recall  f1-score   support

          负面       0.89      0.92      0.90       592
          正面       0.92      0.89      0.90       608

    accuracy                           0.90      1200
   macro avg       0.90      0.90      0.90      1200
weighted avg       0.90      0.90      0.90      1200



In [16]:
textcnn = TextCNN(config)
textcnn.load_model(checkpoint_prefix)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     10527900    input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 198, 64)      57664       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 197, 64)      76864       embedding_2[0][0]                
____________________________________________________________________________________________

In [17]:
textcnn.single_predict("外观很漂亮，出人意料地漂亮，做工非常好") # 单句预测

('正面', 0.9999671)

In [18]:
textcnn.single_predict("书的内容没什么好说的，主要是纸张、印刷太差，所用的纸非常粗糙比一般的盗版书还要差，裁的也不好。") # 单句预测

('负面', 0.9998888)

In [20]:
# LSTM创建

# 定义 LSTM 模型类
class TextLSTM(object):
    def __init__(self, config):
        self.config = config
        self.preprocessor = Preprocessor(config)
        self.class_name = {0: '负面', 1: '正面'}
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)  # 降低学习率

    def build_model(self):
        # 输入层
        idx_input = tf.keras.layers.Input((self.config.max_seq_len,))

        # 嵌入层（增加trainable参数控制）
        input_embedding = tf.keras.layers.Embedding(
            len(self.preprocessor.token2idx),
            self.config.embedding_dim,
            input_length=self.config.max_seq_len,
            mask_zero=True,
            embeddings_regularizer=tf.keras.regularizers.l2(1e-4)  # 添加嵌入层正则化
        )(idx_input)

        # LSTM层
        lstm = tf.keras.layers.LSTM(
            units=128,  # LSTM单元数
            return_sequences=False,  # 只返回最后一个时间步的输出
            kernel_regularizer=tf.keras.regularizers.l2(1e-4)  # LSTM层正则化
        )(input_embedding)

        # 添加批标准化层
        bn = tf.keras.layers.BatchNormalization()(lstm)

        # 全连接层
        dense = tf.keras.layers.Dense(
            64,  # 神经元数量
            activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(0.01)  # 增强L2正则化
        )(bn)

        # 添加Dropout
        dropout = tf.keras.layers.Dropout(0.5)(dense)

        # 输出层
        output = tf.keras.layers.Dense(2, activation='softmax')(dropout)

        # 编译模型
        model = tf.keras.Model(inputs=idx_input, outputs=output)
        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=self.optimizer,  # 使用自定义优化器
            metrics=['accuracy']
        )
        model.summary()
        self.model = model

    def fit(self, x_train, y_train, x_valid=None, y_valid=None, epochs=50, batch_size=64, **kwargs):
        # 初始化回调
        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',  # 改为监控准确率
            patience=10,  # 增大耐心值
            mode='max',
            restore_best_weights=True
        )

        lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6,
            verbose=1
        )

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            './improved_lstm_checkpoint',
            save_weights_only=True,
            save_best_only=True,
            monitor='val_accuracy'
        )

        # 数据预处理
        self.build_model()
        x_train = self.preprocessor.transform(x_train)
        valid_data = (self.preprocessor.transform(x_valid), y_valid) if x_valid is not None else None

        # 合并显式传递的回调和 kwargs 中的回调
        callbacks = [early_stop, lr_scheduler, checkpoint] + kwargs.get('callbacks', [])

        # 过滤掉 kwargs 中的 callbacks 参数
        filtered_kwargs = {k: v for k, v in kwargs.items() if k != 'callbacks'}

        # 开始训练
        history = self.model.fit(
            x=x_train,
            y=y_train,
            validation_data=valid_data,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=callbacks,  # 显式传递合并后的回调
            **filtered_kwargs  # 过滤掉 callbacks 后的其他参数
        )
        return history

    def evaluate(self, x_test, y_test):
        x_test = self.preprocessor.transform(x_test)
        y_pred = np.argmax(self.model.predict(x_test), axis=-1)
        print(classification_report(y_test, y_pred, target_names=['负面', '正面']))

    def single_predict(self, text):
        x = self.preprocessor.transform([text])
        prob = self.model.predict(x)[0]
        label_idx = np.argmax(prob)
        return self.class_name[label_idx], prob[label_idx]

    def load_model(self, ckpt_file):
        self.build_model()
        self.model.load_weights(ckpt_file)



In [21]:
# 初始化LSTM模型并进行训练
textlstm = TextLSTM(config)
textlstm.fit(
    x_train, y_train,
    x_valid=x_valid, 
    y_valid=y_valid,
    epochs=50,
    batch_size=64
)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 200, 300)          10527900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
batch_normalization_4 (Batch (None, 128)               512       
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 130 

/home/ma-user/anaconda3/envs/TensorFlow-2.1/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/ma-user/anaconda3/envs/TensorFlow-2.1/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


9146/9146 [==============================] - 176s 19ms/sample - loss: 2.1950 - accuracy: 0.5704 - val_loss: 1.9680 - val_accuracy: 0.5300
Epoch 2/50
9146/9146 [==============================] - 172s 19ms/sample - loss: 1.5920 - accuracy: 0.8021 - val_loss: 1.6037 - val_accuracy: 0.8267
Epoch 3/50
9146/9146 [==============================] - 171s 19ms/sample - loss: 1.1872 - accuracy: 0.9274 - val_loss: 1.2873 - val_accuracy: 0.8725
Epoch 4/50
9146/9146 [==============================] - 172s 19ms/sample - loss: 0.9633 - accuracy: 0.9680 - val_loss: 1.1159 - val_accuracy: 0.8767
Epoch 5/50
9146/9146 [==============================] - 172s 19ms/sample - loss: 0.8263 - accuracy: 0.9805 - val_loss: 1.0645 - val_accuracy: 0.8675
Epoch 6/50
9146/9146 [==============================] - 173s 19ms/sample - loss: 0.7252 - accuracy: 0.9833 - val_loss: 1.0004 - val_accuracy: 0.8733
Epoch 7/50
9146/9146 [==============================] - 172s 19ms/sample - loss: 0.6297 - accuracy: 0.9882 - val_loss

In [22]:
# 对测试集进行评估
textlstm.evaluate(x_test, y_test)

              precision    recall  f1-score   support

          负面       0.89      0.88      0.88       592
          正面       0.88      0.89      0.89       608

    accuracy                           0.88      1200
   macro avg       0.88      0.88      0.88      1200
weighted avg       0.88      0.88      0.88      1200



In [23]:
# 加载最佳模型权重
textlstm.load_model('./improved_lstm_checkpoint')

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 200, 300)          10527900  
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
batch_normalization_5 (Batch (None, 128)               512       
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 130 

In [26]:
textlstm.single_predict("外观很漂亮，出人意料地漂亮，做工非常好")

('正面', 0.9960388)

In [27]:
textlstm.single_predict("书的内容没什么好说的，主要是纸张、印刷太差，所用的纸非常粗糙比一般的盗版书还要差，裁的也不好。")

('负面', 0.9999999)